In [1]:
!pip install openai
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.6 MB/s eta 0:00:00


In [6]:
import os
import openai
from concurrent.futures import ThreadPoolExecutor, as_completed
import tiktoken
import logging
from tqdm import tqdm
import time
import random
logging.basicConfig(level=logging.INFO) # Setup the logging system
# Paste in your OpenAI key here.
openai.api_key = "sk-....."

In [1]:
from multiprocessing import cpu_count

cpu_count()

2

In [3]:
def load_text(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except Exception as e:
        logging.error(f'Failed to load file {file_path}: {str(e)}')
        raise
def initialize_files(output_file, log_file):
    try:
        open(output_file, 'w').close()
        open(log_file, 'w').close()
    except Exception as e:
        logging.error(f'Failed to initialize files {output_file}, {log_file}: {str(e)}')
        raise
def save_to_file(responses, output_file):
    try:
        with open(output_file, 'w') as file:
            for response in responses:
                file.write(response + '\n')
    except Exception as e:
        logging.error(f'Failed to save to file {output_file}: {str(e)}')
        raise
def log_to_file(log_file, message):
    try:
        with open(log_file, 'a') as file:
            file.write(message + '\n')
    except Exception as e:
        logging.error(f'Failed to log to file {log_file}: {str(e)}')
        raise
def call_openai_api(chunk, model, max_tokens, temperature, prompt):
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {'role': 'system', 'content': prompt},
                    {'role': 'user', 'content': chunk},
                ],
                max_tokens=max_tokens,
                n=1,
                stop=None,
                temperature=temperature,
            )
            return response.choices[0]['message']['content'].strip()
        except openai.error.RateLimitError:
            wait_time = (2 ** i) + random.random()
            logging.warning(f'Rate limit exceeded. Retrying after {wait_time} seconds.')
            time.sleep(wait_time)
        except Exception as e:
            logging.error(f'API call failed: {str(e)}')
            return None
    logging.error('Failed to call OpenAI API after multiple retries due to rate limiting.')
    return None
def split_into_chunks(text, model, tokens=1000):
    encoding = tiktoken.encoding_for_model(model)
    words = encoding.encode(text)
    chunks = []
    for i in range(0, len(words), tokens):
        chunks.append(''.join(encoding.decode(words[i:i + tokens])))
    return chunks

In [4]:
def process_chunks(input_file, output_file, log_file, model, chunksize, max_tokens, temperature, prompt):
    initialize_files(output_file, log_file)
    text = load_text(input_file)
    chunks = split_into_chunks(text, model, tokens=chunksize)
    nCh = len(chunks)
    print(str(nCh) + ' chunks.')
    log_to_file(log_file, f'Number of chunks: {nCh}')
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(call_openai_api, chunk, model, max_tokens, temperature, prompt): chunk for chunk in chunks}
        responses = []
        for future in tqdm(as_completed(futures), total=len(futures), desc='Processing chunks'):
            response = future.result()
            if response is None:
                log_to_file(log_file, f'Failed to process chunk {futures[future]}')
            else:
                responses.append(response)
                log_to_file(log_file, f'Successfully processed chunk!')
    save_to_file(responses, output_file)

Upload an input file to Colab through the left sidebar panel.

In [5]:
input_file = 'input.txt'
output_file = 'output.txt'
log_file = 'log.txt'
model = 'gpt-3.5-turbo-0301' # Can also use gpt-4-0314
chunksize = 100 # This shouldn't be too large (>4000) or OpenAI will be overloaded. A safe size is under 3000 tokens. Your prompt length also counts for the OpenAI token limit.
tokens = 100 # shorter will be faster. but could terminate too early.
temperature = 0.0 # 0.0 is probably best if you are going for highest accuracy
prompt = 'You are an expert Italian English translator. Translate the text to Italian.' # Instructions for GPT. This counts into the 4k token limit.
process_chunks(input_file, output_file, log_file, model, chunksize, tokens, temperature, prompt)

193 chunks.


Processing chunks: 100%|██████████| 193/193 [02:31<00:00,  1.27it/s]
